#### Imports

In [1]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

#!pip install tqdm
from tqdm import tqdm_notebook, tnrange
from itertools import chain

from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

from pylab import imshow

from torchvision import transforms
from torchvision.transforms import functional as TF

from torch.utils.data import Dataset, DataLoader
from torch import save, load


In [2]:
# Set some parameters
im_width = 256
im_height = 256


#### Load the base image and mask

In [3]:
train_dir = "ND2L/t000.tif"
mask_dir = "ND2L/man_seg000.tif"

#### Dataset Class


In [4]:
import pandas as pd 
import os 
from skimage import io
import  csv
from torchvision import datasets, transforms
from PIL import Image
from torch.utils.data import Dataset
import numpy as np

class CellSeg(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir

    def __len__(self):
        return 100 # -------------

    def transform(self, image, mask):

        # Random crop
        i, j, h, w = transforms.RandomCrop.get_params(
            image, output_size=(512, 512))
        image = TF.crop(image, i, j, h, w)
        mask = TF.crop(mask, i, j, h, w)

        # Random horizontal flipping
        if random.random() > 0.5:
            image = TF.hflip(image)
            mask = TF.hflip(mask)

        # Random vertical flipping
        if random.random() > 0.5:
            image = TF.vflip(image)
            mask = TF.vflip(mask)

        # Transform to tensor
        image = TF.to_tensor(image)
        mask = TF.to_tensor(mask)
        return image, mask

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, 't000.tif')
        mask_path = os.path.join(self.mask_dir, 'man_seg000.tif')
        image = Image.open(img_path).convert("RGB")
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
        mask[mask>0] = 255.0
        mask = Image.fromarray(mask)

        x, y = self.transform(image, mask)

        return x, y



#### Create Model

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
     

class Unet(torch.nn.Module):
    def __init__(self, input_ch=3, out_ch=1, feat=[64, 128, 512]):
        super(Unet, self).__init__()

        self.kernel_size = 3
        self.stride = 1
        self.padding_mode = 1
        self.bias = True

        # Contracting
        self.contracting = nn.Sequential(
            
            nn.Conv2d(input_ch, feat[0], self.kernel_size, self.stride, self.padding_mode, self.bias),
            nn.BatchNorm2d(feat[0]),
            nn.ReLU(inplace=True),
            nn.Conv2d(feat[0], feat[0], self.kernel_size, self.stride, self.padding_mode, self.bias),
            nn.BatchNorm2d(feat[0]),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(feat[0], feat[1], self.kernel_size, self.stride, self.padding_mode, self.bias),
            nn.BatchNorm2d(feat[1]),
            nn.ReLU(inplace=True),
            nn.Conv2d(feat[1], feat[1], self.kernel_size, self.stride, self.padding_mode, self.bias),
            nn.BatchNorm2d(feat[1]),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(feat[1], feat[2], self.kernel_size, self.stride, self.padding_mode, self.bias),
            nn.BatchNorm2d(feat[2]),
            nn.ReLU(inplace=True),
            nn.Conv2d(feat[2], feat[2], self.kernel_size, self.stride, self.padding_mode, self.bias),
            nn.BatchNorm2d(feat[2]),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

        )   


        # Fully Connected
        self.connected = nn.Sequential(
            nn.Conv2d(feat[2], feat[2], self.kernel_size, self.stride, self.padding_mode, self.bias),
            nn.BatchNorm2d(feat[2]),
            nn.ReLU(inplace=True),
            nn.Conv2d(feat[2], feat[2], self.kernel_size, self.stride, self.padding_mode, self.bias),
            nn.BatchNorm2d(feat[2]),
            nn.ReLU(inplace=True),
        )
        

        #Expansive -->  Ver conv2d aca
        self.expansive = nn.Sequential(
            
            nn.ConvTranspose2d(feat[2],feat[2], kernel_size=2, stride=2),
            nn.Conv2d(feat[2], feat[2], self.kernel_size, self.stride, self.padding_mode, self.bias),
            nn.BatchNorm2d(feat[2]),
            nn.ReLU(inplace=True),
            nn.Conv2d(feat[2], feat[2], self.kernel_size, self.stride, self.padding_mode, self.bias),
            nn.BatchNorm2d(feat[2]),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(feat[1]*4,feat[1]*4, kernel_size=2, stride=2),
            nn.Conv2d(feat[1]*4, feat[1], self.kernel_size, self.stride, self.padding_mode, self.bias),
            nn.BatchNorm2d(feat[1]),
            nn.ReLU(inplace=True),
            nn.Conv2d(feat[1], feat[1], self.kernel_size, self.stride, self.padding_mode, self.bias),
            nn.BatchNorm2d(feat[1]),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(feat[0]*2,feat[0]*2, kernel_size=2, stride=2),
            nn.Conv2d(feat[0]*2, feat[0], self.kernel_size, self.stride, self.padding_mode, self.bias),
            nn.BatchNorm2d(feat[0]),
            nn.ReLU(inplace=True),
            nn.Conv2d(feat[0], feat[0], self.kernel_size, self.stride, self.padding_mode, self.bias),
            nn.BatchNorm2d(feat[0]),
            nn.ReLU(inplace=True),
        )

        self.final = nn.Conv2d(feat[0], out_ch, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        x = self.contracting(x)
        x = self.connected(x)
        x = self.expansive(x)

        return self.final(x)

def test():
    x = torch.randn((3, 1, 160, 160))
    model = Unet(input_ch=1, out_ch=1)
    preds = model(x)
    assert preds.shape == x.shape

if __name__ == "__main__":
    test()

#### Util Functions

In [6]:
import torch
import torch.nn as nn
from torch.autograd import Variable

from collections import OrderedDict
import numpy as np


def summary(model, input_size, batch_size=-1, device=torch.device('cpu'), dtypes=None, verbose=True):
    result, params_info = summary_string(
        model, input_size, batch_size, device, dtypes)
    if verbose:
        print(result)

    return params_info


def summary_string(model, input_size, batch_size=-1, device=torch.device('cpu'), dtypes=None):
    if dtypes == None:
        dtypes = [torch.FloatTensor]*len(input_size)

    summary_str = ''

    def register_hook(module):
        def hook(module, input, output):
            class_name = str(module.__class__).split(".")[-1].split("'")[0]
            module_idx = len(summary)

            m_key = "%s-%i" % (class_name, module_idx + 1)
            summary[m_key] = OrderedDict()
            summary[m_key]["input_shape"] = list(input[0].size())
            summary[m_key]["input_shape"][0] = batch_size
            if isinstance(output, (list, tuple)):
                summary[m_key]["output_shape"] = [
                    [-1] + list(o.size())[1:] for o in output
                ]
            else:
                summary[m_key]["output_shape"] = list(output.size())
                summary[m_key]["output_shape"][0] = batch_size

            params = 0
            if hasattr(module, "weight") and hasattr(module.weight, "size"):
                params += torch.prod(torch.LongTensor(list(module.weight.size())))
                summary[m_key]["trainable"] = module.weight.requires_grad
            if hasattr(module, "bias") and hasattr(module.bias, "size"):
                params += torch.prod(torch.LongTensor(list(module.bias.size())))
            summary[m_key]["nb_params"] = params
            

        if (
            not isinstance(module, nn.Sequential)
            and not isinstance(module, nn.ModuleList)
        ):
            hooks.append(module.register_forward_hook(hook))

    # multiple inputs to the network
    if isinstance(input_size, tuple):
        input_size = [input_size]

    # batch_size of 2 for batchnorm
    x = [torch.rand(2, *in_size).type(dtype).to(device=device)
         for in_size, dtype in zip(input_size, dtypes)]

    # create properties
    summary = OrderedDict()
    hooks = []

    # register hook
    model.apply(register_hook)

    # make a forward pass
    # print(x.shape)
    model(*x)

    # remove these hooks
    for h in hooks:
        h.remove()

    summary_str += "----------------------------------------------------------------" + "\n"
    line_new = "{:>20}  {:>25} {:>15}".format(
        "Layer (type)", "Output Shape", "Param #")
    summary_str += line_new + "\n"
    summary_str += "================================================================" + "\n"
    total_params = 0
    total_output = 0
    trainable_params = 0
    total_conv2d = 0
    total_linear = 0 
    for layer in summary:
        if 'conv2d' in layer.lower():
            total_conv2d += 1
        if 'linear' in layer.lower():
            total_linear += 1 

        # input_shape, output_shape, trainable, nb_params
        line_new = "{:>20}  {:>25} {:>15}".format(
            layer,
            str(summary[layer]["output_shape"]),
            "{0:,}".format(summary[layer]["nb_params"]),
        )
        total_params += summary[layer]["nb_params"]

        total_output += np.prod(summary[layer]["output_shape"])
        if "trainable" in summary[layer]:
            if summary[layer]["trainable"] == True:
                trainable_params += summary[layer]["nb_params"]
        summary_str += line_new + "\n"

    # assume 4 bytes/number (float on cuda).
    total_input_size = abs(np.prod(sum(input_size, ()))
                           * batch_size * 4. / (1024 ** 2.))
    total_output_size = abs(2. * total_output * 4. /
                            (1024 ** 2.))  # x2 for gradients
    total_params_size = abs(total_params * 4. / (1024 ** 2.))
    total_size = total_params_size + total_output_size + total_input_size

    summary_str += "================================================================" + "\n"
    summary_str += "Total Conv2d layers: {0:,}".format(total_conv2d) + "\n"
    summary_str += "Total Linear layers: {0:,}".format(total_linear) + "\n"
    summary_str += "Total params: {0:,}".format(total_params) + "\n"
    summary_str += "Trainable params: {0:,}".format(trainable_params) + "\n"
    summary_str += "Non-trainable params: {0:,}".format(total_params -
                                                        trainable_params) + "\n"
    summary_str += "----------------------------------------------------------------" + "\n"
    summary_str += "Input size (MB): %0.2f" % total_input_size + "\n"
    summary_str += "Forward/backward pass size (MB): %0.2f" % total_output_size + "\n"
    summary_str += "Params size (MB): %0.2f" % total_params_size + "\n"
    summary_str += "Estimated Total Size (MB): %0.2f" % total_size + "\n"
    summary_str += "----------------------------------------------------------------" + "\n"
    # return summary
    return summary_str, {'total_params': total_params, 
                         'total_trainable_params': trainable_params,
                         'total_conv2d': total_conv2d,
                         'total_linear': total_linear}

In [7]:
def runRamdomSeed():
    torch.manual_seed(0)
    np.random.seed(0)
    random.seed(0)
    # Disabling the benchmarking feature with torch.backends.cudnn.benchmark = False 
    # causes cuDNN to deterministically select an algorithm, possibly at the cost of reduced performance.
    torch.backends.cudnn.benchmark = False 

runRamdomSeed()

def summarize_Unet():
    # Run randomseed here to ensure results are consistent
    runRamdomSeed()
    student_net = Unet()

    # Investigate your network's layers
    # Compare the printed shape with what expected in the specification
    print("\n========= Model summarization ============ ") 
    student_net_info = summary(student_net, (3, 64, 64), device='cpu')


summarize_Unet()


========= Model summarization ============ 
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           1,792
       BatchNorm2d-2           [-1, 64, 64, 64]             128
              ReLU-3           [-1, 64, 64, 64]               0
            Conv2d-4           [-1, 64, 64, 64]          36,928
       BatchNorm2d-5           [-1, 64, 64, 64]             128
              ReLU-6           [-1, 64, 64, 64]               0
         MaxPool2d-7           [-1, 64, 32, 32]               0
            Conv2d-8          [-1, 128, 32, 32]          73,856
       BatchNorm2d-9          [-1, 128, 32, 32]             256
             ReLU-10          [-1, 128, 32, 32]               0
           Conv2d-11          [-1, 128, 32, 32]         147,584
      BatchNorm2d-12          [-1, 128, 32, 32]             256
             ReLU-13          [-1, 128, 32, 32]           

In [8]:
def check_model_exist_by_name(model_name):
    if os.path.exists(model_name + ".pth") and os.path.isfile(model_name + ".pth"):
        print(f"[Successly trained and saved!] Your model named {model_name}.pth has been saved! DO NOT change the file's name! Just include it in your submission files.")
    else:
        print(f"[Successly trained but failed to save!] Your model named {model_name}.pth has not been saved or saved in a diffrent name from what we expect!")
        import glob
        pt_files = glob.glob("*.pth")
        if pt_files:
            print(f"---> Somehow you've saved models as, {pt_files} which is not what autograder expects!")
            print(f"---> We expect that the trained model to be saved exactly as {model_name}.pth")
            print(f"---> What you can do is to manually rename the trained model to be {model_name}.pth")
        else:
            print(f"---> We found no saved models in *.pth format at all! Manually check if your saved the models in other formats or they are not saved at all!")


def save_model(model, name):
    if isinstance(model, Unet):
        return save(model.state_dict(), name + ".pth")
    
    raise ValueError("model type '%s' not supported!"%str(type(model)))


def load_model(name, device_name='cpu'):

    if "." in name:
        name = name.split('.')[0]
        
    if name == "Unet":
        r = Unet()
    else:
        raise ValueError(f"model {name} has not been supported! Check the spelling!")
    r.load_state_dict(load(name + ".pth", map_location=device_name))
    return r

In [9]:
def accuracy(outputs, labels):
    outputs_idx = outputs.max(1)[1].type_as(labels)
    return outputs_idx.eq(labels).float().mean()

def predict(model, inputs, device='cpu'):
    inputs = inputs.to(device)
    logits = model(inputs)
    return F.softmax(logits, -1)

#### Training

In [10]:
def load_data(dataset_path,mask_path, data_transforms=None, num_workers=0, batch_size=128):
    dataset = CellSeg(dataset_path,mask_path)
    return DataLoader(dataset, num_workers=num_workers, batch_size=batch_size, shuffle=True)

In [11]:
class ClassificationLoss(torch.nn.Module):
    def forward(self, input, target):
        return torch.nn.functional.nll_loss(torch.nn.functional.log_softmax(input, dim=-1), target)

In [12]:
class Args(object):
    pass

args = Args();

args.learning_rate = 0.0001
args.log_dir = './my_tensorboard_log_directory' 
args.num_epochs = 5
args.dataset_path = 'ND2L/'
args.mask_path = 'ND2L/'
args.validation_path = 'ND2L/'
args.batch_size = 128

In [13]:
def train(args, model_name="Unet"):
    """
    @Brief: training your model. This should include the following items:
        - Initialize the model (already given). Only need to map the model to the device on which you would want to run the model on 
                using the following syntax: 
                model = model.to(device) 
                where device = torch.device(<device_name>), 
                i.e: device = torch.device("cuda:0") or device = torech.device("cpu")
                    
        - Initialize tensorboard summarizers (already given)
        - Initialize data loaders (you need to code up)
        - Initialize the optimizer (you need to code up. Type is of your choice)
        - Initialize the loss function (you should have coded up above)
        - A for loop to iterate through many epochs (up to your choice). In each epoch:
                - Iterate through every mini-batches (remember to map data and labels to the device that you would want to run the model on)
                        - Run the forward path
                        - Get loss
                        - Calculate gradients 
                        - Update the model's parameters
                - Evaluate your model on the validation set
                - Save the model if the performance on the validation set is better using exactly the following line:
                        save_model(model, model_name) 
                 
    @Inputs: 
        Args: object of your choice to carry arguments that you want to use within your training function. 
    @Output: 
        No return is necessary here. 
    """
    # Do not touch the following lines
    # Initialize the model 
    model = Unet()

    #----------------------------------------
    
    data_loader = load_data(args.dataset_path, args.mask_path, data_transforms=None, num_workers=0, batch_size=args.batch_size)
    criterion = ClassificationLoss()
    optimizer = torch.optim.Adam(model.parameters(), args.learning_rate)

    prev_validation_loss = float('inf')

    for epoch in range(args.num_epochs):
        for i, batch in enumerate(data_loader):
          [X_train, Y_train] = batch
          optimizer.zero_grad()
          outputs = model(X_train)
          loss = criterion(outputs, Y_train)
          loss.backward()
          optimizer.step()
          if i % 20 == 0: print(f'Completed Batch {i} of {int(21002/args.batch_size)}')
        print(f'Completed Epoch {epoch} of {args.num_epochs}')

        val_data_loader = load_data(args.validation_path, batch_size=9001)
        for i, batch in enumerate(val_data_loader):
          optimizer.zero_grad()
          outputs = model(X_train)
          loss = criterion(outputs, Y_train)

          print('Validation Loss', loss.item())

          if loss.item() < prev_validation_loss:
            print(f'Less than previous loss of {prev_validation_loss}, saving model...')
            save_model(model, model_name)
            prev_validation_loss = loss.item()
          print()


    # save_model(model, model_name) 
    # assert os.path.exists(model_name + ".pth") and os.path.isfile(model_name + ".pth"), f"[Fail to save your model named {model_name}.pth!"
    return model

In [ ]:
model_name="Unet"
unet_model = train(args, model_name=model_name)
    
# Make sure that the model you've trained above has already been saved! 
# check_model_exist_by_name(model_name)


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:132: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  img = torch.from_numpy(np.array(pic, np.float32, copy=False))


### Saving Images for Evaluation

In [ ]:
def image_source(image, mask):

    # Random crop
    i, j, h, w = transforms.RandomCrop.get_params(
        image, output_size=(256, 256))
    image = TF.crop(image, i, j, h, w)
    mask = TF.crop(mask, i, j, h, w)

    # Random horizontal flipping
    if random.random() > 0.5:
        image = TF.hflip(image)
        mask = TF.hflip(mask)

    # Random vertical flipping
    if random.random() > 0.5:
        image = TF.vflip(image)
        mask = TF.vflip(mask)

    return image, mask

original_img = Image.open('ND2L/t009.tif')
original_msk = np.array(Image.open('ND2L/man_seg009.tif').convert("L"), dtype=np.float32)
original_msk[original_msk>0] = 255.0
original_msk = Image.fromarray(original_msk)

save_path = 'SavedImgs/'

for i in range(10):
    image, mask = image_source(original_img, original_msk)
    pred = predict(unet_model, img)
    
    image.save(save_path+'im'+str(i)+'.tiff')
    mask.save(save_path+'mas'+str(i)+'.tiff')
    pred.save(save_path+'pred'+str(i)+'.tiff')
